In [1]:
from vcgym.graph.spatial import SpaceGraph
from vcgym.graph.node import Node
from importlib.resources import open_text
import csv

In [2]:
def func():
    return 1

In [3]:
graph = SpaceGraph()
node_ids = ['a', 'b', 'c', 'd'] # <- order matters
edges = [('a', 'b'), ('b', 'c'), ('c', 'd'), ('a', 'c')]

with graph.builder() as gb:
    gb['nodes'] = node_ids
    gb['edges'] = edges
    gb['directed'] = True


with graph._nodes['a'] as node:
    node['name'] = "energy_gen"  
    node['variables']('',0)
    node['functions'](func,('a+','storage')) # + denotes itself at the next time step. 



AttributeError: 'Node' object has no attribute '_graph'

In [ ]:
graph._po

['a', 'b', 'c', 'd']